In [ ]:
import os
import matplotlib.pyplot as plt
import flatbuffers
import tensorflow as tf
import pathlib
import numpy as np
import pandas as pd

import os

from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras import Sequential
from tensorflow.keras.layers import RandomFlip
from tensorflow.keras.layers import RandomRotation
from tensorflow.keras.layers import RandomZoom
from tensorflow.keras.layers import RandomTranslation
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Normalization
from tensorflow.keras.callbacks import LearningRateScheduler

from tflite_support import metadata_schema_py_generated as _metadata_fb
from tflite_support import metadata as _metadata
import math

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:
train_dataset = image_dataset_from_directory(directory="./train", image_size=(224, 224))
val_dataset = image_dataset_from_directory(directory="./val", image_size=(224, 224))
test_dataset = image_dataset_from_directory(directory="./test", image_size=(224, 224))

In [ ]:
class_names = train_dataset.class_names
print(class_names)

In [ ]:
resnet50_base = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [ ]:
resnet50 = ResNet50(weights="imagenet", include_top=True, input_shape=(224, 224, 3))

In [ ]:
inputs = Input(shape=(224, 224, 3))
x = preprocess_input(inputs)
x = resnet50_base(x)
x = Flatten()(x)
outputs = Dense(5, activation="softmax")(x)
transfer_model = Model(inputs=inputs, outputs=outputs)

In [ ]:
resnet50_base.trainable = False

In [ ]:
transfer_model.compile(optimizer=RMSprop(learning_rate=0.0001),
                       loss="sparse_categorical_crossentropy",
                       metrics=["accuracy"])

In [ ]:
transfer_model_history = transfer_model.fit(train_dataset, epochs=15,
                validation_data=val_dataset)

In [ ]:
pd.DataFrame(transfer_model_history.history).plot()

In [ ]:
for layer in resnet50_base.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = False
    else:
        layer.trainable = True

In [ ]:
transfer_model.compile(optimizer=RMSprop(learning_rate=0.001), 
                       loss="sparse_categorical_crossentropy", 
                       metrics=["accuracy"])

In [ ]:
transfer_model_history = transfer_model.fit(train_dataset, epochs=30,
                validation_data=val_dataset)

In [ ]:
pd.DataFrame(transfer_model_history.history).plot()

In [ ]:
path = './horseTest.jpg'

img = tf.keras.utils.load_img(
    path, target_size=(224, 224)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = transfer_model.predict(img_array)

print(
    "This image most likely belongs to {} with a {} percent confidence."
    .format(class_names[np.argmax(predictions[0])], 100 * np.max(predictions[0]))
)

In [ ]:
path = './mocopsisTest.jpg'

img = tf.keras.utils.load_img(
    path, target_size=(224, 224)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = transfer_model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
path = './test4.jpg'

img = tf.keras.utils.load_img(
    path, target_size=(224, 224)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = transfer_model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
# The convnet that was trained and tuned using transfer learning
test_loss, test_acc = transfer_model.evaluate(test_dataset)
test_acc

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(transfer_model)

# quantization, set the optimization mode and data type
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float32]

tflite_model = converter.convert()

# Save the model.
model_file = 'transfer_model.tflite'
with open(model_file, 'wb') as f:
  f.write(tflite_model)